In [14]:
# ============================================
# CONFUSION MATRIX - MA TRẬN NHẦM LẪN
# ============================================
# Đây là bảng đếm để so sánh dự đoán với thực tế
#
# Ví dụ: Dự đoán email SPAM hay KHÔNG SPAM
#
#                   DỰ ĐOÁN
#              |  Spam  | Không spam |
#        ------|--------|------------|
#   T    Spam  |   TP   |     FN     |  <- Thực tế là Spam
#   H    ------|--------|------------|
#   Ự   Không  |   FP   |     TN     |  <- Thực tế Không spam
#   C   spam   |        |            |
#
# 4 giá trị quan trọng:
# - TP (True Positive): Dự đoán Spam, thực tế cũng Spam → ĐÚNG ✓
# - TN (True Negative): Dự đoán Không spam, thực tế cũng Không spam → ĐÚNG ✓
# - FP (False Positive): Dự đoán Spam, nhưng thực tế Không spam → SAI ✗ (báo động giả)
# - FN (False Negative): Dự đoán Không spam, nhưng thực tế là Spam → SAI ✗ (bỏ sót)

def calc(y, y_pred, positive):
    """
    Tính TP, FP, TN, FN từ dữ liệu thực tế và dự đoán
    
    Args:
        y: nhãn thực tế (ground truth) - ['Spam', 'Không spam', ...]
        y_pred: nhãn dự đoán của model - ['Spam', 'Không spam', ...]
        positive: nhãn được coi là "positive" (thường là nhãn quan trọng) - 'Spam'
    
    Returns:
        tp, fp, tn, fn: 4 giá trị của confusion matrix
    """
    tp = 0  # True Positive: dự đoán đúng lớp positive
    fp = 0  # False Positive: dự đoán nhầm thành positive
    tn = 0  # True Negative: dự đoán đúng lớp negative
    fn = 0  # False Negative: bỏ sót lớp positive
    
    # Duyệt qua từng cặp (thực tế, dự đoán)
    for real, pred in zip(y, y_pred):
        if real == pred:  # Nếu dự đoán ĐÚNG
            if real == positive:  # Và là lớp positive
                tp += 1  # → True Positive
            else:  # Và là lớp negative
                tn += 1  # → True Negative
        else:  # Nếu dự đoán SAI
            if real == positive:  # Thực tế là positive nhưng dự đoán sai
                fn += 1  # → False Negative (bỏ sót)
            else:  # Thực tế là negative nhưng dự đoán là positive
                fp += 1  # → False Positive (báo động giả)
    
    return tp, fp, tn, fn

# Ví dụ minh họa:
# Thực tế: ['Spam', 'Không spam', 'Spam']
# Dự đoán: ['Spam', 'Spam', 'Không spam']
# → TP=1 (đoán đúng Spam ở vị trí 0)
# → FP=1 (đoán nhầm Spam ở vị trí 1)
# → FN=1 (bỏ sót Spam ở vị trí 2)
# → TN=0 (không có trường hợp đoán đúng Không spam)

In [15]:
# ============================================
# ACCURACY - ĐỘ CHÍNH XÁC TỔNG THỂ
# ============================================
# Tỷ lệ dự đoán ĐÚNG trên TỔNG SỐ mẫu
#
# Công thức: Accuracy = (TP + TN) / (TP + FP + TN + FN)
#                     = Số dự đoán đúng / Tổng số mẫu
#
# Ví dụ: 10 email, đoán đúng 8 → Accuracy = 8/10 = 0.8 = 80%
#
# ⚠️ Nhược điểm: Không tốt khi dữ liệu mất cân bằng
# VD: 100 email, 95 Không spam, 5 Spam
#     Model đoán tất cả là Không spam → Accuracy = 95% (cao!)
#     Nhưng thực tế bỏ sót HẾT 5 email Spam → Model VÔ DỤNG!

def calc_accuracy(tp, fp, tn, fn):
    """
    Tính độ chính xác tổng thể
    
    Returns: giá trị từ 0 đến 1 (0% đến 100%)
    """
    return (tp + tn) / (tp + fp + tn + fn)

In [ ]:
# ============================================
# PRECISION - ĐỘ CHÍNH XÁC CỦA DỰ ĐOÁN POSITIVE
# ============================================
# Trong số những cái model DỰ ĐOÁN là Positive, có bao nhiêu % là ĐÚNG?
#
# Công thức: Precision = TP / (TP + FP)
#                      = Đúng positive / Tất cả dự đoán positive
#
# Ví dụ: Model đoán 10 email là Spam
#        → Trong đó có 7 email thực sự là Spam (TP=7)
#        → 3 email không phải Spam (FP=3)
#        → Precision = 7/10 = 0.7 = 70%
#
# 💡 Ý nghĩa: "Khi model nói là Spam, tin cậy được bao nhiêu %?"
#
# 🎯 Quan trọng khi: Chi phí FP (báo động giả) cao
# VD: Lọc email → FP = email quan trọng bị bỏ vào Spam → MẤT KHÁCH HÀNG!

def calc_precision(tp, fp):
    """
    Tính độ chính xác của dự đoán positive
    
    Returns: giá trị từ 0 đến 1 (càng cao càng tốt)
    """
    return tp / (tp + fp)

In [16]:
# ============================================
# RECALL (SENSITIVITY) - TỶ LỆ PHÁT HIỆN POSITIVE
# ============================================
# Trong số những cái THỰC TẾ là Positive, model BẮT ĐƯỢC bao nhiêu %?
#
# Công thức: Recall = TP / (TP + FN)
#                   = Bắt được positive / Tất cả positive thực tế
#
# Ví dụ: Có 10 email Spam thực sự
#        → Model phát hiện được 8 email (TP=8)
#        → Bỏ sót 2 email (FN=2)
#        → Recall = 8/10 = 0.8 = 80%
#
# 💡 Ý nghĩa: "Model bắt được bao nhiêu % Spam trong tổng số Spam?"
#
# 🎯 Quan trọng khi: Chi phí FN (bỏ sót) cao
# VD: Phát hiện ung thư → FN = bỏ sót bệnh nhân → NGUY HIỂM TÍNH MẠNG!
#     Phát hiện gian lận → FN = bỏ sót giao dịch lừa đảo → MẤT TIỀN!

def calc_recall(tp, fn):
    """
    Tính tỷ lệ phát hiện positive (còn gọi là Sensitivity)
    
    Returns: giá trị từ 0 đến 1 (càng cao càng tốt)
    """
    return tp / (tp + fn)

In [17]:
# ============================================
# SPECIFICITY - TỶ LỆ PHÁT HIỆN NEGATIVE
# ============================================
# Trong số những cái THỰC TẾ là Negative, model ĐÚNG được bao nhiêu %?
#
# Công thức: Specificity = TN / (TN + FP)
#                        = Đúng negative / Tất cả negative thực tế
#
# Ví dụ: Có 20 email Không spam thực sự
#        → Model đoán đúng 18 email là Không spam (TN=18)
#        → Đoán nhầm 2 email là Spam (FP=2)
#        → Specificity = 18/20 = 0.9 = 90%
#
# 💡 Ý nghĩa: "Model nhận diện đúng Không spam được bao nhiêu %?"
#
# 🎯 Quan trọng khi: Cần tránh báo động giả (FP)
# VD: Test COVID → FP = người khỏe mạnh bị dương tính giả → HOẢNG LOẠN!

def calc_specificity(tn, fp):
    """
    Tính tỷ lệ phát hiện negative
    
    Returns: giá trị từ 0 đến 1 (càng cao càng tốt)
    """
    return tn / (tn + fp)

In [18]:
# ============================================
# F1-SCORE - TRUNG BÌNH ĐIỀU HÒA CỦA PRECISION VÀ RECALL
# ============================================
# Cân bằng giữa Precision (độ chính xác) và Recall (độ phát hiện)
#
# Công thức: F1 = 2 × (Precision × Recall) / (Precision + Recall)
#
# 💡 Tại sao dùng trung bình điều hòa (harmonic mean)?
# - Trung bình thường: (0.9 + 0.1)/2 = 0.5 → Che giấu giá trị thấp!
# - Trung bình điều hòa: 2×0.9×0.1/(0.9+0.1) = 0.18 → Phạt nặng giá trị thấp!
#
# Ví dụ:
# Model A: Precision=0.9, Recall=0.9 → F1=0.90 (cân bằng tốt)
# Model B: Precision=0.9, Recall=0.1 → F1=0.18 (mất cân bằng, F1 thấp!)
#
# 🎯 Khi nào dùng F1?
# - Khi cần cân bằng giữa Precision và Recall
# - Khi dữ liệu mất cân bằng (imbalanced data)
# - Thay thế Accuracy khi Accuracy không đáng tin cậy

def calc_f1score(precision, recall):
    """
    Tính F1-Score (harmonic mean của Precision và Recall)
    
    Returns: giá trị từ 0 đến 1 (càng cao càng tốt)
    """
    return 2 * precision * recall / (precision + recall)

In [19]:
# ============================================
# BALANCED ACCURACY - ĐỘ CHÍNH XÁC CÂN BẰNG
# ============================================
# Trung bình của Recall (phát hiện positive) và Specificity (phát hiện negative)
#
# Công thức: Balanced Acc = (Recall + Specificity) / 2
#
# 💡 Tại sao cần Balanced Accuracy?
# - Accuracy thông thường bị ảnh hưởng bởi dữ liệu mất cân bằng
# - Balanced Accuracy cho trọng số BẰNG NHAU cho cả 2 lớp
#
# Ví dụ: 95 Không spam, 5 Spam
# Model đoán tất cả là Không spam:
# - Accuracy = 95% (cao nhưng VÔ DỤNG!)
# - Recall = 0% (bỏ sót HẾT Spam)
# - Specificity = 100% (đúng hết Không spam)
# - Balanced Acc = (0% + 100%)/2 = 50% (phản ánh đúng model tệ!)
#
# 🎯 Khi nào dùng?
# - Khi dữ liệu mất cân bằng nghiêm trọng
# - Khi cần đánh giá công bằng cả 2 lớp

def calc_balanced_acc(recall, specificity):
    """
    Tính độ chính xác cân bằng
    
    Returns: giá trị từ 0 đến 1 (càng cao càng tốt)
    """
    return (recall + specificity) / 2

In [20]:
# ============================================
# MCC (MATTHEWS CORRELATION COEFFICIENT)
# ============================================
# Hệ số tương quan giữa dự đoán và thực tế, xét trên CẢ 4 giá trị TP, TN, FP, FN
#
# Công thức: MCC = (TP×TN - FP×FN) / √[(TP+FP)(TP+FN)(TN+FP)(TN+FN)]
#
# 💡 Giá trị MCC:
# - MCC = +1: Dự đoán HOÀN HẢO (perfect prediction)
# - MCC = 0: Dự đoán NGẪU NHIÊN (random prediction)
# - MCC = -1: Dự đoán NGƯỢC LẠI HOÀN TOÀN (total disagreement)
#
# 🌟 Ưu điểm MCC:
# - Là chỉ số CÔNG BẰNG NHẤT cho dữ liệu mất cân bằng
# - Xét đến CẢ 4 giá trị của confusion matrix
# - Không bị ảnh hưởng bởi kích thước lớp
#
# Ví dụ so sánh:
# Dataset: 95 Không spam, 5 Spam
# Model đoán tất cả là Không spam:
# - Accuracy = 95% (cao giả tạo!)
# - F1-Score = 0% (vì Precision và Recall = 0)
# - MCC ≈ 0 hoặc âm (phản ánh model tệ!)
#
# 🎯 Khi nào dùng MCC?
# - Khi dữ liệu MẤT CÂN BẰNG nghiêm trọng
# - Khi muốn 1 chỉ số DUY NHẤT đánh giá tổng thể
# - Khi cần so sánh nhiều model khác nhau

import numpy as np

def calc_mcc(tp, fp, tn, fn):
    """
    Tính Matthews Correlation Coefficient
    
    Returns: giá trị từ -1 đến +1
             +1 = hoàn hảo
             0 = ngẫu nhiên
             -1 = tệ nhất
    """
    # Tử số: TP×TN - FP×FN
    numerator = tp * tn - fp * fn
    
    # Mẫu số: √[(TP+FP)(TP+FN)(TN+FP)(TN+FN)]
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    # Tránh chia cho 0
    if denominator == 0:
        return 0
    
    return numerator / denominator

In [21]:
# ============================================
# HÀM BÁO CÁO TỔNG HỢP - IN TẤT CẢ METRICS
# ============================================

def report(y, y_pred, positive):
    """
    Tính và in tất cả các metrics đánh giá model
    
    Args:
        y: nhãn thực tế
        y_pred: nhãn dự đoán
        positive: nhãn được coi là positive
    """
    # Bước 1: Tính confusion matrix
    tp, fp, tn, fn = calc(y, y_pred, positive)
    
    print("="*50)
    print("CONFUSION MATRIX:")
    print(f"TP (True Positive) = {tp}")
    print(f"FP (False Positive) = {fp}")
    print(f"TN (True Negative) = {tn}")
    print(f"FN (False Negative) = {fn}")
    print("="*50)
    
    # Bước 2: Tính các metrics
    precision = calc_precision(tp, fp)
    recall = calc_recall(tp, fn)
    
    print(f'Accuracy (Độ chính xác tổng): {calc_accuracy(tp, fp, tn, fn):.3f}')
    print(f'Precision (Độ chính xác dự đoán +): {precision:.3f}')
    print(f'Recall (Tỷ lệ phát hiện +): {recall:.3f}')
    print(f'Specificity (Tỷ lệ phát hiện -): {calc_specificity(tn, fp):.3f}')
    print(f'F1-Score (Cân bằng P&R): {calc_f1score(precision, recall):.3f}')
    print(f'Balanced Accuracy (Độ chính xác cân bằng): {calc_balanced_acc(recall, calc_specificity(tn, fp)):.3f}')
    print(f'MCC (Hệ số tương quan): {calc_mcc(tp, fp, tn, fn):.3f}')
    print("="*50)

In [22]:
# ============================================
# VÍ DỤ THỰC TẾ: SO SÁNH 3 MODEL LỌC EMAIL SPAM
# ============================================

# Dữ liệu thực tế: 5 email
y = ['Không spam', 'Spam', 'Spam', 'Không spam', 'Spam']
#     Email 0      Email 1  Email 2    Email 3      Email 4

# MODEL A: Dự đoán của model A
ya_pred = ['Không spam', 'Spam', 'Không spam', 'Không spam', 'Spam']
#          ✓ Đúng        ✓ Đúng   ✗ Sai (bỏ sót) ✓ Đúng        ✓ Đúng

# MODEL B: Dự đoán của model B
yb_pred = ['Spam', 'Spam', 'Spam', 'Không spam', 'Không spam']
#          ✗ Sai  ✓ Đúng  ✓ Đúng   ✓ Đúng        ✗ Sai (bỏ sót)

# MODEL C: Dự đoán của model C
yc_pred = ['Không spam', 'Không spam', 'Spam', 'Spam', 'Spam']
#          ✓ Đúng        ✗ Sai (bỏ sót) ✓ Đúng  ✗ Sai   ✓ Đúng

print("📧 ĐÁNH GIÁ MODEL A:")
report(y, ya_pred, 'Spam')

📧 ĐÁNH GIÁ MODEL A:
CONFUSION MATRIX:
TP (True Positive) = 2
FP (False Positive) = 0
TN (True Negative) = 2
FN (False Negative) = 1
Accuracy (Độ chính xác tổng): 0.800
Precision (Độ chính xác dự đoán +): 1.000
Recall (Tỷ lệ phát hiện +): 0.667
Specificity (Tỷ lệ phát hiện -): 1.000
F1-Score (Cân bằng P&R): 0.800
Balanced Accuracy (Độ chính xác cân bằng): 0.833
MCC (Hệ số tương quan): 0.667


In [23]:
print("\n📧 ĐÁNH GIÁ MODEL B:")
report(y, yb_pred, 'Spam')


📧 ĐÁNH GIÁ MODEL B:
CONFUSION MATRIX:
TP (True Positive) = 2
FP (False Positive) = 1
TN (True Negative) = 1
FN (False Negative) = 1
Accuracy (Độ chính xác tổng): 0.600
Precision (Độ chính xác dự đoán +): 0.667
Recall (Tỷ lệ phát hiện +): 0.667
Specificity (Tỷ lệ phát hiện -): 0.500
F1-Score (Cân bằng P&R): 0.667
Balanced Accuracy (Độ chính xác cân bằng): 0.583
MCC (Hệ số tương quan): 0.167


In [24]:
print("\n📧 ĐÁNH GIÁ MODEL C:")
report(y, yc_pred, 'Spam')


📧 ĐÁNH GIÁ MODEL C:
CONFUSION MATRIX:
TP (True Positive) = 2
FP (False Positive) = 1
TN (True Negative) = 1
FN (False Negative) = 1
Accuracy (Độ chính xác tổng): 0.600
Precision (Độ chính xác dự đoán +): 0.667
Recall (Tỷ lệ phát hiện +): 0.667
Specificity (Tỷ lệ phát hiện -): 0.500
F1-Score (Cân bằng P&R): 0.667
Balanced Accuracy (Độ chính xác cân bằng): 0.583
MCC (Hệ số tương quan): 0.167


In [25]:
# ============================================
# TÓM TẮT: KHI NÀO DÙNG METRIC NÀO?
# ============================================
#
# 📊 ACCURACY (Độ chính xác tổng):
# ✅ Dùng khi: Dữ liệu CÂN BẰNG (số lượng các lớp tương đương)
# ❌ KHÔNG dùng khi: Dữ liệu mất cân bằng
#
# 🎯 PRECISION (Độ chính xác dự đoán positive):
# ✅ Dùng khi: Chi phí FALSE POSITIVE cao
# VD: Lọc email → FP = email quan trọng vào spam → mất khách hàng
#     Quảng cáo → FP = gửi quảng cáo cho người không quan tâm → tốn tiền
#
# 🔍 RECALL (Tỷ lệ phát hiện positive):
# ✅ Dùng khi: Chi phí FALSE NEGATIVE cao
# VD: Phát hiện bệnh → FN = bỏ sót bệnh nhân → nguy hiểm tính mạng
#     Phát hiện gian lận → FN = bỏ sót giao dịch lừa đảo → mất tiền
#
# ⚖️ F1-SCORE (Cân bằng Precision & Recall):
# ✅ Dùng khi: Cần cân bằng giữa Precision và Recall
# ✅ Dùng khi: Dữ liệu mất cân bằng (thay thế Accuracy)
#
# 🎲 BALANCED ACCURACY:
# ✅ Dùng khi: Dữ liệu MẤT CÂN BẰNG nghiêm trọng
# ✅ Dùng khi: Muốn đánh giá công bằng CẢ 2 lớp
#
# 🏆 MCC (Matthews Correlation Coefficient):
# ✅ Dùng khi: Cần 1 chỉ số DUY NHẤT, TOÀN DIỆN nhất
# ✅ Dùng khi: Dữ liệu mất cân bằng
# ✅ Dùng khi: So sánh nhiều model khác nhau
# → MCC được coi là METRIC TỐT NHẤT cho binary classification!
#
# ============================================
# QUY TẮC VÀNG:
# ============================================
# 1. Dữ liệu cân bằng → Dùng Accuracy
# 2. Dữ liệu mất cân bằng → Dùng F1-Score hoặc MCC
# 3. FP tốn kém → Tăng Precision (giảm ngưỡng dự đoán)
# 4. FN nguy hiểm → Tăng Recall (tăng ngưỡng dự đoán)
# 5. Không biết dùng gì → Dùng MCC (an toàn nhất!)
#
# ============================================

print("✓ Đã hiểu rõ về các Metrics đánh giá Model!")

✓ Đã hiểu rõ về các Metrics đánh giá Model!
